# PART 4: Decompounded Options

Group Members: 
1. Harshita Sachdev
2. Lu Tinjia
3. Rahul Sreeram
4. Srivatsa Sanjiv Mitragotri
5. Xie Zuoyu
6. Zhang Yonghan
7. Zhao Geping


In [3]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq
from scipy import interpolate
import matplotlib.pylab as plt
import datetime as dt
from scipy.optimize import least_squares
from math import log, sqrt, exp
from scipy.optimize import fsolve
from scipy import integrate

In [4]:
# Fetching the data

fsr = pd.read_csv('fsr.csv', header = 0, index_col = 0)  #fsr is forward swap rate
ois_df = pd.read_csv('ois_result.csv', header = 0)
libor_df = pd.read_csv('LIBOR.csv', header = 0, index_col = 0) 
sabr_df = pd.read_csv('sabr_data.csv', header = 0, index_col = 0) 

In [5]:
sabr_df

,Expiry,Tenor,Alpha,Nu,Rho
0,1,1,0.0068,0.1000,0.2928
1,1,2,0.0102,0.1000,0.2914
2,1,3,0.0112,0.1000,0.3013
3,1,5,0.0096,0.1000,0.4338
4,1,10,0.0104,0.1000,0.4929
5,5,2,0.0118,0.1000,0.2735
6,5,3,0.0122,0.1000,0.2925
7,5,5,0.0111,0.1000,0.3786
8,5,10,0.0119,0.1208,0.8254
9,10,1,0.0122,0.1000,0.1822


In [6]:
ois_df.head()

,Tenor,Rate,Discount Factor
0,0.5,0.002500,0.999993
1,1.0,0.003000,0.997009
2,1.5,0.003125,0.995270
3,2.0,0.003250,0.993531
4,2.5,0.003300,0.991773


#### The IRR derivative equations:

$$
IRR(K) = \sum_{i=1}^{N \cdot m} \frac{1}{m} \cdot \left(1 + \frac{K}{m} \right)^{-i} \quad (\delta = \frac{1}{m})
$$

$$
IRR'(K) = \sum_{i=1}^{N \cdot m} (-i) \cdot \left(\frac{1}{m}\right)^2 \cdot \left(1 + \frac{K}{m} \right)^{-i-1} \quad (\delta = \frac{1}{m})
$$

$$
IRR''(K) = \sum_{i=1}^{N \cdot m} (-i)(-i - 1) \cdot \left(\frac{1}{m}\right)^3 \cdot \left(1 + \frac{K}{m} \right)^{-i-2} \quad (\delta = \frac{1}{m})
$$


In [19]:
def Black76Call(F, K, T, sigma):
    d1 = (np.log(F/K)+(1/2)*(sigma**2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    price = F*norm.cdf(d1) - K*norm.cdf(d2)
    return price

def Black76Put(F, K, T, sigma):
    d1 = (np.log(F/K)+1/2*(sigma**2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    price = K*norm.cdf(-d2) - F*norm.cdf(-d1)
    return price
    return disc*(K*norm.cdf(-d2) - S*norm.cdf(-d1))

def SABR(F, K, T, alpha, beta, rho, nu):
    X = K #ATM
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*log(F/X)
        zhi = log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sigma = numer/denom

    return sigma


def IRR(K, tenor, delta):
    sum = 0
    for i in np.arange(1, tenor/delta + 1):
        sum += delta*(1 + K*delta)**(-i)
    return sum

def IRR1(K, tenor, delta):
    sum = 0
    for i in np.arange(1, tenor/delta + 1):
        sum += (-i)*(delta**2)*(1 + K*delta)**(-i-1)
    return sum

# second derivative
def IRR2(K, tenor, delta):
    sum = 0
    for i in np.arange(1, tenor/delta + 1):
        sum += (-i)*(-i-1)*(delta**3)*(1 + K*delta)**(-i-2)
    return sum

### Qn 1:
**A *decompounded option* pays the following at time \( T = 5y \):**

$$
\text{CMS } 10y^{1/p} - 0.04^{1/q}
$$

where \( p = 4 \) and \( q = 2 \). Use static replication to value the PV of this payoff.


In [22]:
def g(K):
    return K**0.25 - 0.2

def g1(K):
    return 0.25*K**(-0.75)

def g2(K):
    return (-3/16)*K**(-7/4)

def h(K, tenor, delta):
    return g(K)/IRR(K, tenor, delta)

def h1(K,  tenor, delta):
    nominator = IRR(K, tenor, delta)*g1(K) - g(K)*IRR1(K, tenor, delta)
    denominator = IRR(K, tenor, delta)**2
    return nominator/denominator

def h2(K, tenor, delta):
    nominator = IRR(K, tenor, delta)*g2(K) - g(K)*IRR2(K, tenor, delta) - 2*IRR1(K, tenor, delta)*g1(K)
    denominator = IRR(K, tenor, delta)**2
    term2 = 2*IRR1(K, tenor, delta)**2*g(K)/IRR(K, tenor, delta)**3
    return nominator/denominator + term2

In [23]:
D = ois_df[ois_df['Tenor'] == 5]['Discount Factor'].iloc[0] 


fsr_5_10 = fsr[(fsr['Expiry']== '5Y') & (fsr['Tenor'] == '10Y')]
# print(fsr_5_10)
F = fsr_5_10['Forward Swap Rate'].values[0]
print(F)
beta = 0.9

alpha = 0.0119
rho = 0.8254
nu = 0.1208

tenor = 10
delta = 0.5
T = 5

V_rec = integrate.quad(lambda K: h2(K, tenor, delta)*Black76Put(F, K, T, SABR(F, K, T, alpha, beta, rho, nu)), 0, F)
V_pay = integrate.quad(lambda K: h2(K, tenor, delta)*Black76Call(F, K, T, SABR(F, K, T, alpha, beta, rho, nu)), F, 1000)

pv = D* g(F) + V_rec[0] + V_pay[0]
pv

0.0422048766500685


np.float64(0.24873983886530626)

## Qn 2

Suppose the payoff is now

$$
\left( \text{CMS } 10y^{1/p} - 0.04^{1/q} \right)^+
$$

Use static replication to value the PV of this payoff.


In [28]:
tenor = 10
delta = 2
T = 5
L = 0.2**4

term1 = h1(L, tenor, delta)*Black76Call(F, L, T, SABR(F, L, T, alpha, beta, rho, nu))
term2 = integrate.quad(lambda K: h2(K, tenor,delta)*Black76Call(F, K, T, SABR(F, K, T, alpha, beta, rho, nu)), L,1000)
PV_caplet = term1+ term2[0]
PV_caplet

np.float64(0.1281109803825053)